<a href="https://colab.research.google.com/github/pallavmarch/WebScraping_PsychologyToday/blob/main/psychologytoday_webscraping_dataset_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
pd.set_option("display.max_columns", None)
df=pd.read_csv("/content/therapists_profiles_data_Texas.csv",encoding='latin-1')

In [33]:
df['Age'] = df['Age'].replace({'Elders (65+)': 'Elders'})

In [35]:
#df['expertise'] = df['expertise'].astype(str)
df['Dementia'] = df['Expertise'].str.lower().str.contains('dementia')
df['Anxiety'] = df['Expertise'].str.lower().str.contains('anxiety')
df['Depression'] = df['Expertise'].str.lower().str.contains('depression')
df['Elders (65+)']=df['Age'].str.lower().str.contains('elders')
df.head()


,Profile URL,Name,Title,Location,State,ZIP Code,Phone,Individual Sessions ($),Couple Sessions ($),Pay By,Insurance,Top Specialties,Expertise,Types of Therapy,Personal Statement,Verified by Psychology Today,In Practice (years),Membership,Certificate,Attended University,Major/Degree,Graduation Year,Qualifications,Endorsement Count,Endorsed By,Age,Participants,Communities,Religion,I also speak,Dementia,Anxiety,Depression,Elders (65+)
0,https://www.psychologytoday.com/us/therapists/...,Mario Christian Salgado,"Licensed Professional Counselor,MA,LPC","San Antonio, TX 78232(830) 988-4854",TX,78232,(830) 988-4854,$140,Not Found,"ACH Bank transfer, American Express, Discover,...",Aetna | BlueCross and BlueShield | Cigna and E...,Not Found,Anger Management | Anxiety | Body Positivity |...,Art Therapy | Cognitive Behavioral (CBT) | Exp...,"Hello, my name is Mario! I want you all to get...",Yes,4,Not Found,Not Found,Texas A&M University,MA - Clinical Mental Health Counseling,2020,Verified byPsychology TodayLicensed by State o...,2,"Kayla Davis - Licensed Professional Counselor,...",Children (6 to 10) | Preteen | Teen | Adults,Individuals,Not Found,Not Found,Not Found,False,True,True,False
1,https://www.psychologytoday.com/us/therapists/...,Denis 'Woodja' Flanigan,"Psychologist,PhD","2211 Norfolk StSuite 206Houston, TX 77098(713)...",TX,77098,(713) 999-3640,$215,$215,"American Express, Cash, Check, Discover, Healt...",Out of Network,Grief | Anxiety | Relationship Issues,Bisexual | Career Counseling | Codependency | ...,Cognitive Behavioral (CBT) | Culturally Sensit...,We can feel the most alone when we are hurting...,Yes,Not Found,Not Found,Not Found,University of Florida,Not Found,2011,Verified byPsychology TodayExpiredLicensed by ...,0,Not Found,Teen | Adults,Individuals | Couples,Bisexual Allied | Gay Allied | HIV / AIDS Alli...,Other Spiritual or Religious Affiliations,Not Found,False,False,True,False
2,https://www.psychologytoday.com/us/therapists/...,Carol Burnett-Williams,"Licensed Professional Counselor,LPC(she, her)","Houston, TX 77082(346) 512-4081",TX,77082,(346) 512-4081,Not Found,Not Found,Not Found,Aetna | BlueCross and BlueShield | Cigna and E...,Anxiety | Coping Skills | Depression,ADHD | Alcohol Use | Anger Management | Behavi...,Acceptance and Commitment (ACT) | Child/Parent...,"My name is Carol B. Williams, and I am a licen...",Yes,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Verified byPsychology TodayLicensed by State o...,0,Not Found,Preteen | Teen | Adults,Individuals | Couples | Family,Bisexual Allied | Gay Allied | Intersex Allied...,Not Found,Not Found,False,False,False,False
3,https://www.psychologytoday.com/us/therapists/...,"Wenona Andress, LCSW-S","Clinical Social Work/Therapist,LCSW","Wenona Andress, LCSWLaguna Vista, TX 78578(806...",TX,78578,(806) 474-5787,$60,$80,"Cash, Discover, Health Savings Account, Master...",Aetna | BlueCross and BlueShield | Cigna and E...,Chronic Pain | Mood Disorders | Stress,Addiction | Addictive Family Systems | ADHD | ...,Christian Counseling | Clinical Supervision an...,Do you have chronic pain and wonder if you can...,Yes,14,Not Found,University of Massachusetts Center-Tobacco Res...,University of Houston Graduate School of Socia...,Not Found,2005,Verified byPsychology TodayExpiredLicensed by ...,1,Robert Brown Fortney - Licensed Professional C...,Adults | Elders (65+),Individuals | Family,Single Mother,Christian,Not Found,False,False,True,True
4,https://www.psychologytoday.com/us/therapists/...,Clarissa Anahi Cantu,"Licensed Professional Counselor,MS,LPC,LPCC","Austin, TX 78701(512) 640-0239",TX,78701,(512) 640-0239,$150,$170,"Mastercard, Paypal, Venmo, Visa, Zelle",Aetna | BlueCross and BlueShield | Cigna and E...,Depression | Anxiety | Grief,Anger Management | Behavioral Issues | Bisexua...,Adlerian | Cognitive Behavioral (CBT) | Compas...,"Hi there, I am Clarissa! I know it takes a lot...",Yes,7,Not Found,EMDR Therapy Basic Training2020,University of Texas

In [37]:
df[['Dementia','Elders (65+)']].value_counts()

Dementia  Elders (65+)
False     False           4773
          True            4101
True      True             170
          False             34
Name: count, dtype: int64